# Movie Rating Hypothesis testing

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going prove of disprove a hypothesis

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from IPython.display import HTML, display

# pd.set_option('display.notebook_repr_html', False)
# pd.set_option('display.max_columns', 20)
# pd.set_option('display.max_rows', 25)

In [4]:
%matplotlib inline

##### Read and cleanup the movie data

In [5]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames, encoding='latin-1')
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames, encoding='latin-1')
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames, encoding='latin-1')

    return users, ratings, movies

users, ratings, movies = get_movie_data()

tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

print(movies.head())

C:\Users\p_bud\AppData\Local\Temp\ipykernel_26344\2599110875.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table(os.path.join('../data','users.dat'),
C:\Users\p_bud\AppData\Local\Temp\ipykernel_26344\2599110875.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table(os.path.join('../data', 'ratings.dat'),


   movie_id                               title                        genres  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

   year                  short_title  
0  1995                    Toy Story  
1  1995                      Jumanji  
2  1995             Grumpier Old Men  
3  1995            Waiting to Exhale  
4  1995  Father of the Bride Part II  


C:\Users\p_bud\AppData\Local\Temp\ipykernel_26344\2599110875.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table(os.path.join('../data', 'movies.dat'),


### 1. Prove or disprove the following Null Hypothesis:

#### H0: Different age segments of users rate the different genres of movies similarly

- Age segments are (0-20), (21-30), (31-50), (50 and above),
- Genres - Drama, Comedy, Action, Romance, Adventure
- Rating 4 or above is a positive, else negative (used for binary votes)

#### Solution

#### Merge the datasets 

In [6]:
#merge the tables on user_id as we need the ratings and the movies associated with it
merge = pd.merge( users , pd.merge(ratings, movies , on = 'movie_id') , on = 'user_id')

In [7]:
merge

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres,year,short_title
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
1,1,F,1,10,48067,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,1996,James and the Giant Peach
2,1,F,1,10,48067,914,3,978301968,My Fair Lady (1964),Musical|Romance,1964,My Fair Lady
3,1,F,1,10,48067,3408,4,978300275,Erin Brockovich (2000),Drama,2000,Erin Brockovich
4,1,F,1,10,48067,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,1998,"Bug's Life, A"
...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,M,25,6,11106,1150,5,956715569,"Return of Martin Guerre, The (Retour de Martin...",Drama,1982,"Return of Martin Guerre, The (Retour de Martin..."
1000205,6040,M,25,6,11106,2751,1,956716438,From the Hip (1987),Comedy,1987,From the Hip
1000206,6040,M,25,6,11106,3289,5,956704305,Not One Less (Yi ge dou bu neng shao) (1999),Drama,1999,Not One Less (Yi ge dou bu neng shao)
1000207,6040,M,25,6,11106,722,3,960971992,"Haunted World of Edward D. Wood Jr., The (1995)",Documentary,1995,"Haunted World of Edward D. Wood Jr., The"


#### We will try to create labels for age as 'Below 20','Between 21 and 30','Between 31 and 50','Above 50'. This will help us get the count for each of these labels created. For this I am using pd.cut() function 

* https://pandas.pydata.org/docs/reference/api/pandas.cut.html

In [8]:
#create age category by using the pd.cut() function
merge['age_category'] = pd.cut(merge.age, [0,20,30,50,120] , labels = ['Below 20','Between 21 and 30','Between 31 and 50','Above 50'])

#### Create the binary labels for rating column where rating above 4 is a Positive label and below 4 is a negative label. While creating the label I gave the range 0 to 3.99 as I want 4 rating to be positive. 

In [9]:
#creating labels for ratings in.e 
merge['rating_category'] = pd.cut(merge.rating, [0,3.99,5] , labels = ['Negative','Positive'])

In [10]:
merge

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres,year,short_title,age_category,rating_category
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest,Below 20,Positive
1,1,F,1,10,48067,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,1996,James and the Giant Peach,Below 20,Negative
2,1,F,1,10,48067,914,3,978301968,My Fair Lady (1964),Musical|Romance,1964,My Fair Lady,Below 20,Negative
3,1,F,1,10,48067,3408,4,978300275,Erin Brockovich (2000),Drama,2000,Erin Brockovich,Below 20,Positive
4,1,F,1,10,48067,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,1998,"Bug's Life, A",Below 20,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,M,25,6,11106,1150,5,956715569,"Return of Martin Guerre, The (Retour de Martin...",Drama,1982,"Return of Martin Guerre, The (Retour de Martin...",Between 21 and 30,Positive
1000205,6040,M,25,6,11106,2751,1,956716438,From the Hip (1987),Comedy,1987,From the Hip,Between 21 and 30,Negative
1000206,6040,M,25,6,11106,3289,5,956704305,Not One Less (Yi ge dou bu neng shao) (1999),Drama,1999,Not One Less (Yi ge dou bu neng shao),Between 21 and 30,Positive
1000207,6040,M,25,6,11106,722,3,960971992,"Haunted World of Edward D. Wood Jr., The (1995)",Documentary,1995,"Haunted World of Edward D. Wood Jr., The",Between 21 and 30,Negative


#### Now, I am trying to filter the data with genre equal to 'Drama' ,'Comedy', 'Action', 'Romance', 'Adventure'

In [11]:
data = merge[merge['genres'].isin(['Drama' ,'Comedy', 'Action', 'Romance', 'Adventure'])]

### Since we are interested only in the Positive rating we will filter data only with positive values. 

In [12]:
data = data[data['rating_category'] == 'Positive']
data

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres,year,short_title,age_category,rating_category
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest,Below 20,Positive
3,1,F,1,10,48067,3408,4,978300275,Erin Brockovich (2000),Drama,2000,Erin Brockovich,Below 20,Positive
12,1,F,1,10,48067,2398,4,978302281,Miracle on 34th Street (1947),Drama,1947,Miracle on 34th Street,Below 20,Positive
13,1,F,1,10,48067,2918,4,978302124,Ferris Bueller's Day Off (1986),Comedy,1986,Ferris Bueller's Day Off,Below 20,Positive
15,1,F,1,10,48067,2791,4,978302188,Airplane! (1980),Comedy,1980,Airplane!,Below 20,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000200,6040,M,25,6,11106,80,5,957717093,"White Balloon, The (Badkonake Sefid ) (1995)",Drama,1995,"White Balloon, The (Badkonake Sefid )",Between 21 and 30,Positive
1000203,6040,M,25,6,11106,263,4,960971893,Ladybird Ladybird (1994),Drama,1994,Ladybird Ladybird,Between 21 and 30,Positive
1000204,6040,M,25,6,11106,1150,5,956715569,"Return of Martin Guerre, The (Retour de Martin...",Drama,1982,"Return of Martin Guerre, The (Retour de Martin...",Between 21 and 30,Positive
1000206,6040,M,25,6,11106,3289,5,956704305,Not One Less (Yi ge dou bu neng shao) (1999),Drama,1999,Not One Less (Yi ge dou bu neng shao),Between 21 and 30,Positive


### The function pd.crosstab()  acts a frequency table and calculates for each genre and age what is the positive and counts of ratings. 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html


#### We are interested in understanding how the age and genres are related in rating a movie so we will consider only the two columns to determine the expected and the observed values for a chi - square test. 

In [13]:
genre_age_table = pd.crosstab(data.genres,data.age_category, margins = True)

In [14]:
genre_age_table

age_category,Below 20,Between 21 and 30,Between 31 and 50,Above 50,All
genres,,,,,
Action,1070,2359,2315,235,5979
Adventure,94,286,598,104,1082
Comedy,14685,25642,19957,2009,62293
Drama,12784,27233,28224,4454,72695
Romance,175,359,470,37,1041
All,28808,55879,51564,6839,143090


#### The above table obtained is a set of observed values and we slice the table  ignoring the total count. 

In [19]:
observed_values = genre_age_table.iloc[0:5,0:4]

In [20]:
observed_values

age_category,Below 20,Between 21 and 30,Between 31 and 50,Above 50
genres,,,,
Action,1070,2359,2315,235
Adventure,94,286,598,104
Comedy,14685,25642,19957,2009
Drama,12784,27233,28224,4454
Romance,175,359,470,37


#### Now we calculate the expected values and this can be obtained from the stats.chi2_contingency() function. This function computes the chi-square statistic and p-value for the hypothesis test of independence of the observed frequencies in the contingency table shown above.

In [21]:
value = stats.chi2_contingency(observed_values)

In [22]:
value

(2096.282344430687,
 0.0,
 12,
 array([[ 1203.73912922,  2334.89790342,  2154.59610036,   285.76686701],
        [  217.83671815,   422.53880774,   389.9101824 ,    51.7142917 ],
        [12541.31486477, 24326.44172898, 22447.94361591,  2977.29979034],
        [14635.52701097, 28388.5939269 , 26196.41470403,  3474.4643581 ],
        [  209.58227689,   406.52763296,   375.1353973 ,    49.75469285]]))

In [23]:
expected_val = value[3]

### The degrees of freedom for a test of independence equals the product of the number of categories in each variable minus 1. Thus we calculate by using the formula df = (no_rows - 1) * (no_cols - 1) as shown in class notebook.  Also, from stats.chi2_contingency() we see that value of df is 12 

#### There are 5 rows and 4 columns so our degree of freedom will be 4 * 3 

In [24]:
#calculated the rows and columns from the table
no_rows = len(genre_age_table.iloc[0:5,0])
no_cols = len(genre_age_table.iloc[0,0:4])
df = (no_rows - 1) * (no_cols - 1)

In [25]:
print(df)

12


### calculating the chi square statistic from the formula. 

Referred the below site to calculate the value.

    http://localhost:8888/notebooks/CSC-605_Fall_2022/Class_Resources/Lecture_06/Statistics/06_Chi-Squared%20Test.ipynb

In [26]:
chi_squared_stat = (((observed_values-expected_val)**2)/expected_val).sum().sum()
print(chi_squared_stat)

2096.282344430686


In [27]:
from scipy.stats import chi2
critical_val = chi2.ppf(q = 0.95 , df = df)
print(critical_val)

21.02606981748307


In [28]:
#calculate the p-value
pval = 1 - chi2.cdf(x=chi_squared_stat,df=df)
print(pval)

0.0


In [29]:
if chi_squared_stat >= critical_val:
    print("Reject Null hypothesis between the two variables as the chi-square statistics is greater than the critical value")
else:
    print("Accept the Null Hypthesis")
    
if pval <= 0.05:
    print("Reject the Null Hypothesis as the pvalue is less than 0.05")
else:
    print("Accept the Null Hypothesis")

Reject Null hypothesis between the two variables as the chi-square statistics is greater than the critical value
Reject the Null Hypothesis as the pvalue is less than 0.05


#### Inference: 

From the above test we see that there is no similarity between the age groups and genre for rating a movie. 
Differenet age group have different taste for movie. The young crowd would like action movies where as the elder once might prefer romance. So here age and genre are independant variable and do not have any relation between them. 

The chi square test is performed when the variables are not related to each other and used to test the null hypothesis. Thus based on the above values we can conclude that the null hypothesis is rejected if the critical value is less than the chi square statistic. Also we check for pvalue, if the pvalue is less than 0.05 we reject the Null  hypothesis.